<a href="https://colab.research.google.com/github/cyss93/CP1_team8/blob/%EC%98%A4%EC%B0%BD%EC%98%81/Summer_products_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score
from IPython.display import display
from sklearn.model_selection import train_test_split

from matplotlib import style
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import Ridge


import time
import itertools

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('summer-products-with-rating-and-performance_2020-08.csv')

- 변수 정리
 'title' = 
 'title_orig',
 'price',
 'retail_price',
 'currency_buyer',
 'units_sold',
 'uses_ad_boosts',
 'rating',
 'rating_count',
 'rating_five_count',
 'rating_four_count',
 'rating_three_count',
 'rating_two_count',
 'rating_one_count',
 'badges_count',
 'badge_local_product',
 'badge_product_quality',
 'badge_fast_shipping',
 'tags',
 'product_color',
 'product_variation_size_id',
 'product_variation_inventory',
 'shipping_option_name',
 'shipping_option_price',
 'shipping_is_express',
 'countries_shipped_to',
 'inventory_total',
 'has_urgency_banner',
 'urgency_text',
 'origin_country',
 'merchant_title',
 'merchant_name',
 'merchant_info_subtitle',
 'merchant_rating_count',
 'merchant_rating',
 'merchant_id',
 'merchant_has_profile_picture',
 'merchant_profile_picture',
 'product_url',
 'product_picture',
 'product_id',
 'theme',
 'crawl_month'

In [3]:
df.isna().sum()

title                              0
title_orig                         0
price                              0
retail_price                       0
currency_buyer                     0
units_sold                         0
uses_ad_boosts                     0
rating                             0
rating_count                       0
rating_five_count                 45
rating_four_count                 45
rating_three_count                45
rating_two_count                  45
rating_one_count                  45
badges_count                       0
badge_local_product                0
badge_product_quality              0
badge_fast_shipping                0
tags                               0
product_color                     41
product_variation_size_id         14
product_variation_inventory        0
shipping_option_name               0
shipping_option_price              0
shipping_is_express                0
countries_shipped_to               0
inventory_total                    0
h

In [4]:
# 결측값 있는 행 제거
# has_urgency_banner, urgency_text, merchant_profile_picture 이 세 컬럼은 결측값의 비율이 너무 높기때문에 drop
df.drop(columns = ['has_urgency_banner', 'urgency_text', 'merchant_profile_picture'], inplace = True)

In [5]:
# 결측값 있는 행 제거
df.dropna(axis = 0, inplace = True)

In [6]:
df.isna().sum()

title                           0
title_orig                      0
price                           0
retail_price                    0
currency_buyer                  0
units_sold                      0
uses_ad_boosts                  0
rating                          0
rating_count                    0
rating_five_count               0
rating_four_count               0
rating_three_count              0
rating_two_count                0
rating_one_count                0
badges_count                    0
badge_local_product             0
badge_product_quality           0
badge_fast_shipping             0
tags                            0
product_color                   0
product_variation_size_id       0
product_variation_inventory     0
shipping_option_name            0
shipping_option_price           0
shipping_is_express             0
countries_shipped_to            0
inventory_total                 0
origin_country                  0
merchant_title                  0
merchant_name 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1457 entries, 0 to 1572
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   title                         1457 non-null   object 
 1   title_orig                    1457 non-null   object 
 2   price                         1457 non-null   float64
 3   retail_price                  1457 non-null   int64  
 4   currency_buyer                1457 non-null   object 
 5   units_sold                    1457 non-null   int64  
 6   uses_ad_boosts                1457 non-null   int64  
 7   rating                        1457 non-null   float64
 8   rating_count                  1457 non-null   int64  
 9   rating_five_count             1457 non-null   float64
 10  rating_four_count             1457 non-null   float64
 11  rating_three_count            1457 non-null   float64
 12  rating_two_count              1457 non-null   float64
 13  rat

In [8]:
# 제외 변수들
# 첫번째 제외 변수: title, title_orig, currency_buyer, tags, product_color, product_variation_size_id
# 제외 변수중 고려해볼 만 한 변수
## tags(인코딩 활용. 인코딩 활용시 컬럼이 과하게 늘어난다. 주의), product_variation_size_id
## shipping_option_name: 프랑스어와 아랍어? 등 뒤죽박죽
## urgency_text: 품절임박! 같은 건데, 프랑스어로 되어있다. 일단은 빼고 이후에 원핫인코딩해서 넣어보자 (missing 값이 1100개이다. 유의할 것)
## origin_country: 이것도 원핫인코딩해서 적용 가능
## merchant_title: 판매자 이름 우선 제외
## merchant_name, merchant_id: 위와 동일
## merchant_info_subtitle: 판매자 정보 글. 일단 제외. 배지 여부와 다른 변수로도 판단 가능?
## merchant_profile_picture: 판매자 사진 링크
## product_url, product_picture: 링크 데이터
## product_id: 제품 키
## theme: 해당 제품들을 찾기위해 검색한 검색어. summer로 모두 동일해서 제외
## crawl_month: 크롤링 날짜라서 제외
## has_urgency_banner: 결측값이 많아서 일단 제외 (결측값이 아마 없다는 뜻?) (1000개 이상의 결측값)
## rating_five_count + rating_four_count + rating_three_count + rating_two_count + rating_one_count : 결측값이 많음 (36개)

In [9]:
# 상수항 추가. 실행이 될 때마다 추가되기 때문에 독립된 셀로 위에 빼두었습니다.
df = sm.add_constant(df,has_constant="add")

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [10]:
# 다중모델 학습을 위한 특성
features = ['price',
            'retail_price',
            'uses_ad_boosts',
            'rating',
            'rating_count',
            'rating_five_count',
            'rating_four_count',
            'rating_three_count',
            'rating_two_count',
            'rating_one_count',
            'badges_count',
            'badge_local_product',
            'badge_product_quality',
            'badge_fast_shipping',
            'product_variation_inventory',
            'shipping_option_price',
            'shipping_is_express',
            'countries_shipped_to',
            'inventory_total',
            'merchant_rating_count',
            'merchant_rating',
            'merchant_has_profile_picture',
            'const']

In [11]:
target = 'units_sold'
x = df[features]
y = df[target]

# 분할
X_train ,X_test , y_train , y_test = train_test_split(x,y,train_size=0.7,test_size=0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1019, 23) (438, 23) (1019,) (438,)


In [12]:
#Train the MLR / 회귀모델적합
Model = sm.OLS(y_train, X_train)
FittedModel = Model.fit()

print(FittedModel.summary())

                            OLS Regression Results                            
Dep. Variable:             units_sold   R-squared:                       0.812
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     215.3
Date:                Tue, 05 Jul 2022   Prob (F-statistic):               0.00
Time:                        10:17:44   Log-Likelihood:                -9857.9
No. Observations:                1019   AIC:                         1.976e+04
Df Residuals:                     998   BIC:                         1.986e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
price           

- 각 변수들의 p-value 값이 0.8, 0.6 이상이 되는 값들이 보이기에 다중공선성이 의심됩니다(통상 0.5이상이라면 다중공선성 확인). 또한 모델 summary 결과에서도 다중공선성(multicollinearity)이 의심된다는 결과가 나왔기에 VIF(분산팽창지수)를 확인해보았습니다. 

In [13]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif["features"] = X_train.columns
print(vif)

/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


     VIF Factor                      features
0      4.580933                         price
1      1.139066                  retail_price
2      1.052294                uses_ad_boosts
3      1.223069                        rating
4           inf                  rating_count
5           inf             rating_five_count
6           inf             rating_four_count
7           inf            rating_three_count
8           inf              rating_two_count
9           inf              rating_one_count
10          inf                  badges_count
11          inf           badge_local_product
12          inf         badge_product_quality
13          inf           badge_fast_shipping
14     1.237303   product_variation_inventory
15     4.496709         shipping_option_price
16     1.283536           shipping_is_express
17     1.060418          countries_shipped_to
18     1.031599               inventory_total
19     1.261943         merchant_rating_count
20     1.224694               merc

- vif값이 10이 넘으면 다중공선성을 의심합니다. 여기선 7개만을 제외하고 모두 10이 넘고, inf값까지 나온것을 볼 수 있습니다. 그렇기에 해당 값들을 drop하려 합니다.

In [14]:
# 다중공선성 제외 1
# 다중공선성 제외 특성
features_after_vif = ['price',
                      'retail_price',
                      'uses_ad_boosts',
                      'rating_count',
                      'badge_product_quality',
                      'badge_fast_shipping',
                      'product_variation_inventory',
                      'countries_shipped_to',
                      'merchant_rating_count',
                      'merchant_rating',
                      'merchant_has_profile_picture',
                      'const']


target = 'units_sold'
x = df[features_after_vif]
y = df[target]

# 분할
X_train ,X_test , y_train , y_test = train_test_split(x,y,train_size=0.7,test_size=0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#Train the MLR / 회귀모델적합
Model = sm.OLS(y_train, X_train)
FittedModel = Model.fit()

print(FittedModel.summary())

# 에러
y_pred = FittedModel.predict(X_test)

# 평가지표 계산
print('평가지표')
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
display(pd.DataFrame([['MSE', mse],['MAE', mae],['RMSE', rmse],['R2', r2]], columns=['Metric', 'Score']))


# VIF 계산
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif["features"] = X_train.columns
print(vif)

(1019, 12) (438, 12) (1019,) (438,)
                            OLS Regression Results                            
Dep. Variable:             units_sold   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     486.3
Date:                Tue, 05 Jul 2022   Prob (F-statistic):               0.00
Time:                        10:17:44   Log-Likelihood:                -9794.7
No. Observations:                1019   AIC:                         1.961e+04
Df Residuals:                    1007   BIC:                         1.967e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------

,Metric,Score
0,MSE,1.898223e+07
1,MAE,1.879053e+03
2,RMSE,4.356860e+03
3,R2,7.609577e-01


    VIF Factor                      features
0     1.212587                         price
1     1.114237                  retail_price
2     1.038258                uses_ad_boosts
3     1.096998                  rating_count
4     1.061602         badge_product_quality
5     1.048265           badge_fast_shipping
6     1.200191   product_variation_inventory
7     1.060598          countries_shipped_to
8     1.101910         merchant_rating_count
9     1.125937               merchant_rating
10    1.032787  merchant_has_profile_picture
11  490.240749                         const


- vif값을 토대로 변수들을 제거, 추가하며 모델링 작업을 진행하였습니다. 과정에서 평가지표를 전후로 비교하며 최적의 모델 변수들을 찾고자 하였고, 위 모델에서 모델 설명력과 평가지표, vif값들 모두 양호하게 도출되었습니다.

### 변수 선택
- 변수선택법으로는 전진선택법을 선택하였습니다.
- 선택기준으로, AIC, BIC 등 중에서 실제값의 분포를 보려는 것이 아니라 예측을 하는 것이니 AIC를 선택하였습니다.
- 관련 링크 
  - https://aliencoder.tistory.com/24
  - https://zephyrus1111.tistory.com/65

In [15]:
# 변수선택을 위해서 타겟과 features가 함께 있는데이터로 구하자
select = features
select.append('units_sold')

df_for_select = df[select]

In [16]:
def forward_feature_selection(X_train, y_train, variables=X_train.columns.tolist() ):
    import statsmodels.api as sm
    import matplotlib.pyplot as plt
    import warnings
    warnings.filterwarnings("ignore")
    
    y = y_train ## 반응 변수

    selected_variables = [] ## 선택된 변수들
    sl_enter = 0.5
    sl_remove = 0.5
    
    sv_per_step = [] ## 각 스텝별로 선택된 변수들
    adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
    steps = [] ## 스텝
    step = 0
    while len(variables) > 0:
        remainder = list(set(variables) - set(selected_variables))
        pval = pd.Series(index=remainder) ## 변수의 p-value
        ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
        ## 선형 모형을 적합한다.
        for col in remainder: 
            X = X_train[selected_variables+[col]]
            X = sm.add_constant(X)
            model = sm.OLS(y,X).fit(disp=0)
            pval[col] = model.pvalues[col]
    
        min_pval = pval.min()
        if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
            selected_variables.append(pval.idxmin())
            ## 선택된 변수들에대해서
            ## 어떤 변수를 제거할지 고른다.
            while len(selected_variables) > 0:
                selected_X = X_train[selected_variables]
                selected_X = sm.add_constant(selected_X)
                selected_pval = sm.OLS(y,selected_X).fit(disp=0).pvalues[1:] ## 절편항의 p-value는 뺀다
                max_pval = selected_pval.max()
                if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                    remove_variable = selected_pval.idxmax()
                    selected_variables.remove(remove_variable)
                else:
                    break
            
            step += 1
            steps.append(step)
            adj_r_squared = sm.OLS(y,sm.add_constant(X_train[selected_variables])).fit(disp=0).rsquared_adj
            adjusted_r_squared.append(adj_r_squared)
            sv_per_step.append(selected_variables.copy())
        else:
            break
    return selected_variables

In [17]:
selected_features = forward_feature_selection(x, y)

In [18]:
selected_features

['rating_count',
 'const',
 'retail_price',
 'merchant_rating_count',
 'badge_fast_shipping',
 'uses_ad_boosts',
 'product_variation_inventory',
 'badge_product_quality',
 'countries_shipped_to',
 'price']

In [19]:
# 웹에서의 순서를 고려하여, 순서 임의 변경
selected_features = ['rating_count',
                      'const',
                      'retail_price',
                      'merchant_rating_count',
                      'badge_fast_shipping',
                      'uses_ad_boosts',
                      'product_variation_inventory',
                      'badge_product_quality',
                      'countries_shipped_to',
                      'price',
                      'merchant_rating']

In [20]:
# 전진선택법으로 선택한 feature들로 모델링
model_2 = sm.OLS(y_train, sm.add_constant(pd.DataFrame(X_train[selected_features]))).fit(disp=0)
print(model_2.summary())


# 에러
y_pred = model_2.predict(X_test[selected_features])

# 평가지표 계산
print('평가지표')
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
display(pd.DataFrame([['MSE', mse],['MAE', mae],['RMSE', rmse],['R2', r2]], columns=['Metric', 'Score']))


# VIF 계산
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train[selected_features].values,i) for i in range(X_train[selected_features].shape[1])]
vif["features"] = X_train[selected_features].columns
print(vif)

                            OLS Regression Results                            
Dep. Variable:             units_sold   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     535.4
Date:                Tue, 05 Jul 2022   Prob (F-statistic):               0.00
Time:                        10:17:45   Log-Likelihood:                -9794.7
No. Observations:                1019   AIC:                         1.961e+04
Df Residuals:                    1008   BIC:                         1.967e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
rating_count      

,Metric,Score
0,MSE,1.898091e+07
1,MAE,1.878737e+03
2,RMSE,4.356709e+03
3,R2,7.609742e-01


    VIF Factor                     features
0     1.087913                 rating_count
1   487.047331                        const
2     1.114180                 retail_price
3     1.098242        merchant_rating_count
4     1.046401          badge_fast_shipping
5     1.038250               uses_ad_boosts
6     1.200187  product_variation_inventory
7     1.061566        badge_product_quality
8     1.060143         countries_shipped_to
9     1.211963                        price
10    1.115780              merchant_rating


#### 전진선택법을 적용했을 때, 결국 같은 변수들을 선택하는 결과를 보였다. 그렇기에 모델1이 적합한 모델이라고 생각하였다. (전진선택법을 한 model_2가 결국 기존 모델을 만들게 됨)

- 선택 변수 설명
  - price: 가격
  - retail_price: 할인 가격
  - uses_ad_boosts: 광고 유무
  - rating_count: 받은 별점 수
  - badge_product_quality: 품질 보증 배지 유뮤
  - badge_fast_shipping: 빠른 배송 배지 유무
  - product_variation_inventory: 재고량
  - countries_shipped_to: 배송가능 국가 수
  - inventory_total: 제품의 모든 재고량(사이즈, 색깔 모두)
  - merchant_rating_count: 판매자의 별점 수
  - merchant_rating: 판매자 별점
  - merchant_has_profile_picture: 판매자 프로필사진 유무